<a href="https://colab.research.google.com/github/AnikethDandu/traffic-sign-classification/blob/main/TrafficSignClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convolutional Neural Network class

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
  def __init__(self):
    # Input image: 32x32x3
    self.PADDING_SIZE = 1
    self.KERNEL_SIZE = 3
    self.STRIDE = 1
    self.POOL_SIZE = 2
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv2 = nn.Conv2d(32, 64, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv3 = nn.Conv2d(64, 128, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.conv4 = nn.Conv2d(128, 256, 
                           kernel_size=self.KERNEL_SIZE, 
                           stride=self.STRIDE, 
                           padding=self.PADDING_SIZE)
    self.fc1 = nn.Linear(2304, 512)
    self.fc2 = nn.Linear(512, 43)
    
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv2(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv3(x)), self.POOL_SIZE)
    x = F.max_pool2d(F.relu(self.conv4(x)), self.POOL_SIZE)
    x = x.flatten(start_dim=1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


    

In [ ]:
import numpy as np
import cv2
import os

class DataProcessing():
  def __init__(self):
    self.IMG_SIZE = 50
    self.CLASS_COUNT = 43
    self.TRAIN_PATH = 'traffic_sign_images/Train'
    self.TEST_PATH = 'traffic_sign_images/Test'
    self.testing_data = []
    self.training_data = []
    self.training_folder_paths = []

  def make_train_set(self):
    self.training_folder_paths = [self.TRAIN_PATH + f'/{str(i)}' for i in range(self.CLASS_COUNT)]
    for path in self.training_folder_paths:
      for image in os.listdir(path):
        image_class = int(self.training_folder_paths.index(path))
        image = cv2.imread(os.path.join(path, image), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (self.IMG_SIZE, self.IMG_SIZE))
        self.training_data.append([np.array(image), np.eye(self.CLASS_COUNT)[image_class]])
    np.random.shuffle(self.training_data)
    np.save('training_data.npy', self.training_data, allow_pickle=True)

  def make_test_set(self):
    class_labels = {}
    with open('traffic_sign_images/Test.csv', 'r') as file:
      for line in file.readlines():
        line_array = line.split(',') 
        if line_array[0] != 'Width':
          image_index = line_array[7][-10:-5].lstrip('0')
          class_labels[image_index] = int(line_array[6])
    for image in os.listdir(self.TEST_PATH):
      if image[0] != 'G':
        image_class = class_labels[image[0:5].lstrip('0')]
        image = cv2.imread(os.path.join(self.TEST_PATH, image), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (self.IMG_SIZE, self.IMG_SIZE))
        self.testing_data.append([np.array(image), np.eye(self.CLASS_COUNT)[image_class]])
    np.save('testing_data.npy', self.testing_data, allow_pickle=True)


    

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
import numpy as np

EPOCHS = 10
BATCH_SIZE = 32

training_data = []
testing_data = []

image_size = (-1, 3, 50, 50)

DataProccesor = DataProcessing()
if 'training_data.npy' not in os.listdir():
  DataProccesor.make_train_set()
if 'testing_data.npy' not in os.listdir():
  DataProccesor.make_test_set()
training_data = np.load('training_data.npy', allow_pickle=True)
testing_data = np.load('testing_data.npy', allow_pickle=True)

total_images = 0
class_img_count = []
for folder in os.listdir(DataProccesor.TRAIN_PATH):
  if folder != '.DS_Store':
    image_count = len([img for img in os.listdir(os.path.join(DataProccesor.TRAIN_PATH, folder))])
    class_img_count.append(image_count)
    total_images += image_count

FINAL_WEIGHTS = torch.Tensor([1 - img_count/total_images for img_count in class_img_count])
CNN = ConvNet()
OPTIMIZER = optim.Adam(CNN.parameters(), lr=0.002)
CRITERION = nn.CrossEntropyLoss(weight=FINAL_WEIGHTS)

TRAIN_IMAGES = torch.tensor([i[0] for i in tqdm(training_data)]).view(image_size)
TRAIN_IMAGES = TRAIN_IMAGES / 255.0
TRAIN_LABELS = torch.tensor([i[1] for i in tqdm(training_data)], dtype=torch.float32)

TEST_IMAGES = torch.tensor([i[0] for i in tqdm(testing_data)]).view(image_size)
TEST_IMAGES = TEST_IMAGES / 255.0
TEST_LABELS = torch.tensor([i[1] for i in tqdm(testing_data)], dtype=torch.float32)

for epoch in range(EPOCHS):
  for i in tqdm(range(0, len(training_data), BATCH_SIZE)):
    BATCH_IMGS = TRAIN_IMAGES[i:i + BATCH_SIZE].view(image_size)
    BATCH_LABELS = TRAIN_LABELS[i:i + BATCH_SIZE]

    OPTIMIZER.zero_grad()
    OUTPUTS = CNN(BATCH_IMGS)
    LOSS = CRITERION(OUTPUTS, torch.tensor([list(label).index(1) for label in BATCH_LABELS]).long())
    LOSS.backward()
    OPTIMIZER.step()
  print(f'Epoch: {epoch}, Loss: {LOSS}')

with torch.no_grad():
  total_classes = {}
  class_correct = {}
  total_images = 0
  total_correct = 0 
  for i in tqdm(range(len(TEST_IMAGES))):
    CORRECT_CLASS = torch.argmax(TEST_LABELS[i])
    PREDICTED_CLASS = torch.argmax(CNN(TEST_IMAGES[i].view(image_size))[0])
    
    total_images += 1
    total_classes[PREDICTED_CLASS.item()] = total_classes[PREDICTED_CLASS.item()] + 1 if PREDICTED_CLASS in total_classes else 1
    
    if PREDICTED_CLASS == CORRECT_CLASS:
      total_correct += 1
      class_correct[CORRECT_CLASS.item()] = class_correct[CORRECT_CLASS.item()] + 1 if CORRECT_CLASS in class_correct else 1
print([f'Accuracy for {img_class}: {round(100 * class_correct[img_class] / total_classes[img_class], 3)}%' for img_class in class_correct])
print(f'Total accuracy: {round(100 * total_correct / total_images, 3)}%')



In [ ]:
'''
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/gdrive/My\ Drive/ColabNotebooks/Data/ traffic_sign_images.zip
!unzip traffic_sign_images.zip/traffic_sign_images.zip
!rm -r traffic_sign_images.zip/
'''